<a href="https://colab.research.google.com/github/MelihKlc/Deep-Learning/blob/main/Transfer_learning_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Transfer learning with fine-tuning

In [12]:
# We use script for all helper function.
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-08-06 14:57:12--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-08-06 14:57:12 (103 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [13]:
 # Import helper functions we are going to use in this notebook
 from helper_functions import create_tensorboard_callback , plot_loss_curves , unzip_data , walk_through_dir

# Lets get some data

This time we are going to see how we can use the pretrained models within tf.keras.applications and then apply them to our own problem (recognizing images of food)
link : https://www.tensorflow.org/api_docs/python/tf/keras/applications


In [14]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

--2023-08-06 14:57:12--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.119.128, 108.177.126.128, 108.177.127.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.119.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip.1’

10_food_classes_10_ 100%[===================>] 160.74M  41.1MB/s    in 4.5s    

2023-08-06 14:57:17 (35.5 MB/s) - ‘10_food_classes_10_percent.zip.1’ saved [168546183/168546183]



In [15]:
unzip_data("10_food_classes_10_percent.zip")

In [16]:
#Lets check out how many images and subdirectories are in our dataset
walk_through_dir("10_food_classes_10_percent")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/test'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/sushi'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ramen'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/steak'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/fried_rice'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_curry'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/hamburger'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ice_cream'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ch

In [17]:
# Create training and test directory path
train_dir = "/content/10_food_classes_10_percent/train"
test_dir = "/content/10_food_classes_10_percent/test"

#We will use different class. This class is faster than ImageDataGenerator. Yani bu sefer datagen olusturmamıza gerek olmayacak direkt bu fonksiyonun icinde halledicez hepsini.
import tensorflow as tf
train_data = tf.keras.preprocessing.image_dataset_from_directory(directory = train_dir ,
                                                                 batch_size = 32 ,
                                                                 image_size = (224,224) ,
                                                                 label_mode = "categorical"
                                                                 )
test_data = tf.keras.preprocessing.image_dataset_from_directory(directory = test_dir ,
                                                                batch_size = 32 ,
                                                                image_size = (224,224) ,
                                                                label_mode = "categorical"
                                                                )

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [18]:
train_data  #Batch dataset olarak geldi tensor. None kısmı aslında batch size. 10 classes

<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>

In [19]:
# Check out the classnames our dataset
train_data.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [20]:
# See an example of a batch of data
for images , labels in train_data.take(1): #take bizim batchimizi unravel(çözmek) etmemizi sağlıyor. Girdiğimiz 1 de 1 batch al demek
  print(images , labels)

tf.Tensor(
[[[[3.53265305e+01 2.53265305e+01 2.43265305e+01]
   [3.81428566e+01 2.81428585e+01 2.71428585e+01]
   [4.07091827e+01 2.87091827e+01 2.87091827e+01]
   ...
   [8.02092972e+01 2.11918442e+02 1.35275650e+02]
   [8.19744339e+01 2.13377502e+02 1.37377502e+02]
   [8.21277008e+01 2.13301224e+02 1.38714462e+02]]

  [[3.39285736e+01 2.39285717e+01 2.19285717e+01]
   [3.69897995e+01 2.69897957e+01 2.59897957e+01]
   [3.45000000e+01 2.25000000e+01 2.25000000e+01]
   ...
   [7.79388046e+01 2.05984711e+02 1.30540878e+02]
   [8.32653275e+01 2.11280640e+02 1.36275528e+02]
   [8.73571777e+01 2.15433716e+02 1.40408203e+02]]

  [[3.17244911e+01 2.15102043e+01 1.95102043e+01]
   [3.46275520e+01 2.36275520e+01 2.16275520e+01]
   [3.42142868e+01 2.23826542e+01 2.20459194e+01]
   ...
   [7.99795761e+01 2.05979584e+02 1.32193863e+02]
   [7.94132996e+01 2.05413300e+02 1.31627594e+02]
   [8.12856445e+01 2.07285645e+02 1.33499924e+02]]

  ...

  [[1.38521576e+01 2.28521576e+01 3.78521576e+01]
   [1

# Model 0 : Building a transfer learning feature extraction model using the Keras functional API

The sequential API is straightforward , it runs our layers in sequential order.

But the functional API gives us more flexibility with our models

In [23]:
# 1. Create a base model with tf.keras.applications
base_model = tf.keras.applications.EfficientNetB0(include_top = False)  #Top layer aslında output layer oluyor biz bunu değiştireceğimiz için false yapıyoruz.

# 2. Freeze the base model ( so the underlying pre-trained patterns arent updated)
base_model.trainable = False

# 3. Create inputs into our model
inputs = tf.keras.layers.Input(shape = (224,224,3) , name = "input_layer")

# 4. If using resnet50v2 you will need to normalize inputs. But in efficientnetb0 , we dont have to.
# x = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)  eğer normalize yapmamız gerekirse böyle yapıyoruz.

# 5. Pass the inputs to the base model
x = base_model(inputs)  #base model inputlarımızı alıyor bu şekilde
print(f"Shape after passing inputs through base model: {x.shape}")

# 6. Average pool the outputs of the base model (aggregate all the most important informations , reduce number of computations)
x = tf.keras.layers.GlobalAveragePooling2D(name = "global_average_pooling_layer")(x)  #aslında bu x i sona koydugumuzda yukarda olusturdugumuz x = basemodel kısmını buraya baglamıs oluyoruz.
print(f"Shape after GlobalAveragePooling2D : {x.shape}")

# 7. Create the output activation layer
outputs = tf.keras.layers.Dense(10 , activation = "softmax" , name = "output_layer")(x)

# 8. Combine the inputs with the outputs into a model
model_0 = tf.keras.Model(inputs , outputs)

# 9. Compile a model

model_0.compile(loss = ["categorical_crossentropy"] ,
                optimizer = tf.keras.optimizers.Adam() ,
                metrics = ["accuracy"]
                )
# 10. Fit the model

hist_0 = model_0.fit( train_data , epochs = 5 , steps_per_epoch = len(train_data) , validation_data = test_data , validation_steps = int( 0.25 * len(test_data)) , callbacks = [create_tensorboard_callback(dir_name = "transfer_learning" , experiment_name = "10_percent_feature_extractor") ]) # test datanın %25 ini alıyoruz


Exception ignored in: <function _xla_gc_callback at 0x7effd6dc32e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 103, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


Shape after passing inputs through base model: (None, 7, 7, 1280)
Shape after GlobalAveragePooling2D : (None, 1280)
Epoch 1/5


KeyboardInterrupt: ignored

In [ ]:
# Evaluate on the full test dataset
model_0.evaluate(test_data)

In [ ]:
# Check the layers our base model
for layer_number , layer in enumerate(base_model.layers):
  print(layer_number , layer.name)

In [ ]:
base_model.summary()

In [ ]:
model_0.summary()

In [ ]:
plot_loss_curves(hist_0)

# Getting a feature vector from a trained model   ( (None , 1280) aslında bu bizim feature vectorumuz. None kısmı batch size )

Lets get the global average pooling 2d layer

We have a tensor after our model goes through 'base_model' of shape (None , 7 , 7 , 1280).

But then , when it passes through GlobalAveragePooling2D , it turns into (None , 1280).

Lets use a similar shaped tensor of (1 , 4 , 4 , 3) and then pass it to GlobalAveragePooling2D